# Exploring IPV6
<i>Adapted for use with FABRIC from [Exploring  IPv6](https://www.cs.unc.edu/Research/geni/geniEdu/10-IPv6.html)</i>
    
In this tutorial you will experiment with IPV6.
    
<b> Prerequisites  
    
* You need to have your FABRIC bastion host key pair set up to do this tutorial. If you have not already set this up, follow steps 1-5 on the first section at https://github.com/fabric-testbed/teaching-materials/blob/main/Getting%20Started.md#section-1-get-started.
* You are comfortable using ssh and executing basic commands using a UNIX shell. [Tips about how to login to hosts.](https://github.com/fabric-testbed/teaching-materials/blob/main/Getting%20Started.md#section-1-get-started)
    
This is the second step in this assignment. To go to the previous step go to slice creation notebook or click [Here](./CreateSlice.ipynb)
    
 In the 1990s, it was realized that the 32 bits allotted for IPV4 addressing would quickly run out. The Internet Engineering Task Force (IETF) designed IPV6 to solve this problem. IPV6 uses 128 bits for addressing, allowing for up to 2<sup>128</sup> unique addresses. Transitioning to IPV6 is slow, and thus, IPV4 functionality is maintained. To deal with the coexistance you can either dual stack or tunnel. The dual stack method allows for devices to handle both protocols while tunneling places the IPV6 packet in the data of IPV4 during transit on older protocols.
    
Addresses are written in 8 sets of 4 hexadecimal digits as so:
    <br>`fe80:ab70:0000:0000:0000:0000:0f68:9786`
    
The addresses are abbreviated by not writing leading zeros and cutting out a 4 digit set of 0s and replacing with ::. 
Thus, the above address becomes this:
    <br>`fe80:ab70::f68:9786`
    
    Generally the first 3 sets of bits are used for global routing, 1 set for internal routing and 4 sets for node addresses.
    
<img src="https://github.com/fabric-testbed/teaching-materials/blob/main/assignments/Exploring%20IPV6/figures/add.png?raw=true">

## 1. Retrieve Slice
Create the slice at the [Create Slice Notebook](./CreateSlice.ipynb) and import it here.
#### Import the Fabric API

In [ ]:
# Load Fablib and Node Information
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()                    
fablib.show_config()
import json
import traceback

slice_name = "IPV6"
slice = fablib.get_slice(slice_name)
slice.list_nodes()

## 2. Guided Experiment

<img src="https://github.com/fabric-testbed/teaching-materials/blob/main/assignments/Exploring%20IPV6/figures/IPV6Top.png?raw=true">

### 2.1 Basic Connection
We will be connecting to a node using IPV6.
1. Open a terminal to all three nodes.

2. On `host1` run:
    <br>`ifconfig`
    <br> We can see the addresses of the interfaces. Take note of the loopback address (lo). The IPV4 address should be `127.0.0.1`, and the IPV6 address should be `::1`.
    
3. On `host1` ping `host1`:
    <br>Using IPV4: `ping 127.0.0.1 -c 5`
    <br>Using IPV6: `ping6 ::1 -c 5`
4. Ping the multicast address to discover all link local neighbors on `host1`:
<br>`ping6 -I <link-name> ff02::1 -c 105`
<br>Where `<link-name>` is the name of the interface on a network (ex: enp7s0). This command will discover all link local neighbors. In this example, the output will be from the router node.

5. Ping the router node from `host1`:
<br>`ping6 -I <link-name> <IP6-address> -c 5`
<br>The link used must be specified as in IPV6, link local addressing does not involve routing.
    
6. If you try to ping `host2` from `host1` you will see it fails as you are using link local addressing and have not advertised any additional routes.
### 2.2 Examine Headers
We will examine the difference between IPV4 and IPV6 packets.
1. On `host1` startup tcpdump using:
<br>`sudo tcpdump -v -i <outgoing interface>`
<br> Where the `<outgoing interface>` is the interface we will be grabbing traffic from (found with `ifconfig`).
    
2. On `router`  ping `host1` using IPV4:
<br>`ping 10.1.1.1 -c 5`
<br> Take note of the output on `node1`.
    
3. On `router`  ping `host1` using IPV6:
<br>`ping6 -I <outgoing link name> fe80::1 -c 5`
<br>Where `<outgoing link name>` is the link connected to `node1`.
    
4. Take a look at the differences in packets caught by tcmpdump:
    * IPV6 has a "flow" field, intended to be used for time important transfers
    * IPV6 headers have a standard 40 bytes. If options are needed they are pointed to in the next header field
    
    * Field names have been updated to better reflect their use
    * IPV6 doesn't allow for fragmentation
    
### 2.3 Use a 'Global' Type Address
We will be assigning global type interfaces to the nodes so that they can be pinged without specifying an interface.

1. Choose an address for `host1` (suggested to use the range `2001:0db8::/32` which has been set aside for tutorials and documentation) and run:
<br>`sudo ip -6 addr add <ip6-address>/64 dev <outgoing-link-name>`
<br> Remember that global unicast addresses begin with 001 in the first three bits. Additionally, do not worry about overlap as this address is not facing the internet.

2. Ping the address you just set from `host1` (pinging `host1`):
<br>`ping6 <ip6-address> -c 5`

3. The address isn't advertised yet. Verify on `host1` with:
<br>`ip -6 route`
4. Setup route to be advertised. On `host1`:
<br>`sudo ip route add <truncated-ip6-address>/64 dev <outgoing-link-name>`
<br>An example of: `<truncated-ip6-address>` if you chose `2001:db8::1` as your address, then you would use `2001:db8::/64` as your truncated address
5. Run through steps 1-4 on the `router` node using a different ip.
6. Try pinging `host1` from the `router` using the new addresses.

## 3.  Assignment
### Part 1
Take a look at part 2.3 in the Tutorial. Set up the third node with a global address and set the routing information so you can ping freely between the nodes. If turning this in for a grade, specify what commands you used and why, and what the resulting routing tables looked like.
### Part 2
Return to section 2.2. Repeat the experiment and see what other differences you can find in the headers. See if you can match each piece of the header you find to the pieces in the diagrams of the two header types below. You will probably not see each field come up in the tcpdump output.

<img src="https://github.com/fabric-testbed/teaching-materials/blob/main/assignments/Exploring%20IPV6/figures/10-IPv4Header.png?raw=true">
<br>
<img src="https://github.com/fabric-testbed/teaching-materials/blob/main/assignments/Exploring%20IPV6/figures/10-IPv6Header.png?raw=true">

If you are turning this in for an assignment, then answer the following questions.

1. What other major differences do you notice between the two headers?
2. IPv6 does not include a checksum field. Why is this field not necessary? In what major cases does this save time?
3. What are the advantages of a fixed length header over the variable length header?
4. Why might IPv6 not support fragmentation?
5. If an IPv6 packet is translated to an IPv4 packet (translation approach) instead of wrapped inside one (tunneling approach), what information would be lost?

## Cleanup Resources
Once you have completed the assignment shut down the slice.

In [ ]:
# Delete Slice
try:
    #To delete the slice change "CHECK" to "True", this is to prevent accidental slice deletion
    CHECK = False
    if (CHECK):
        slice = fablib.get_slice(slice_name)
        slice.delete()
    else:
        print("Change the Boolean to delete slice")
except Exception as e:
    print(f"Fail: {e}")